In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt

Data Cleaning Checklist:
- Remove Duplicates – Check for and drop duplicate rows.
- Handle Missing Values – Identify NaN values and decide whether to fill them or drop the rows/columns.
- Standardize Formats – Ensure dates, numbers, and strings are in a consistent format.
- Trim Whitespaces – Clean up extra spaces in text fields.
- Correct Data Types – Convert columns to their appropriate types (e.g., strings to dates, floats to integers).
- Remove Special Characters – Especially in text or numerical fields that should be clean.
- Check for Outliers – Identify extreme values that might be errors.
- Normalize/Categorize Data – Standardize categorical values (e.g., "NY" vs. "New York").
- Validate Data – Ensure values fall within expected ranges.
- Check for Duplicated or Incorrect Entries in Key Columns – Especially if you have primary keys or important identifiers.

- do both dates columns
- check verification_status, why for some lines it is broken?
- do from and to destination
- 1: "Very Poor", 2: "Poor", 3: "Neutral", 4: "Good", 5: "Excellent" with mapping

In [2]:
df = pd.read_csv('skytrax-data.csv')
df.head()

,date_published,review_text,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended
0,19th March 2025,✅Trip Verified| Flight mainly let down by a ...,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,4.0,4,yes
1,16th March 2025,✅Trip Verified| Another awful experience by ...,Not informed,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,1.0,1,no
2,16th March 2025,"✅Trip Verified| The service was rude, full o...",Not informed,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,5.0,1,no
3,16th March 2025,✅Trip Verified| This flight was a joke. Ther...,Not informed,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,1.0,1,no
4,7th March 2025,✅Trip Verified| This time British Airways ma...,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,5.0,5,yes


In [3]:
df.dtypes

date_published                  object
review_text                     object
aircraft                        object
type_of_traveller               object
seat_type                       object
route                           object
date_flown                      object
seat_comfort_ranking           float64
cabin_staff_service_ranking    float64
food_beverages_ranking         float64
ground_services_ranking        float64
value_for_money                  int64
recommended                     object
dtype: object

In [4]:
df.isnull().sum()

date_published                   0
review_text                      0
aircraft                         0
type_of_traveller                0
seat_type                        0
route                            0
date_flown                     774
seat_comfort_ranking           126
cabin_staff_service_ranking    141
food_beverages_ranking         438
ground_services_ranking        846
value_for_money                  0
recommended                      0
dtype: int64

In [5]:
df.loc[df["review_text"].str.startswith(("✅", "❎"), na=False), "review_text"] = df["review_text"].str[1:]

In [6]:
df[["verification_status", "review_text_clean"]] = df["review_text"].str.split(r'\|\s+', n=1, expand=True)

In [7]:
df.loc[df["review_text_clean"].isna(), "review_text_clean"] = df["verification_status"]
df.loc[df["verification_status"] == df["review_text_clean"], "verification_status"] = "Not informed"

In [8]:
conditions = [
    df['verification_status'].str.contains('Not Verified', case=False, na=False),
    df['verification_status'].str.contains('Trip Verified', case=False, na=False),
]

choices = ['Not Verified', 'Trip Verified']


df['verification_status'] = np.select(conditions, choices, default='Not informed')

In [9]:
df = df.drop(columns = 'review_text')
df.head()

,date_published,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended,verification_status,review_text_clean
0,19th March 2025,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,4.0,4,yes,Trip Verified,Flight mainly let down by a disagreeable fligh...
1,16th March 2025,Not informed,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,1.0,1,no,Trip Verified,Another awful experience by British Airways. T...
2,16th March 2025,Not informed,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,5.0,1,no,Trip Verified,"The service was rude, full of attitude to me, ..."
3,16th March 2025,Not informed,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,1.0,1,no,Trip Verified,This flight was a joke. There was four people ...
4,7th March 2025,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,5.0,5,yes,Trip Verified,This time British Airways managed to get every...


In [10]:
df['date_published'] = df['date_published'].str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
df['date_published'] = pd.to_datetime(df['date_published'], format="%d %B %Y", errors='coerce')
df.head()

,date_published,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended,verification_status,review_text_clean
0,2025-03-19,Boeing 787,Solo Leisure,Business Class,Cape Town to London,March 2025,5.0,2.0,5.0,4.0,4,yes,Trip Verified,Flight mainly let down by a disagreeable fligh...
1,2025-03-16,Not informed,Business,Business Class,London to Geneva,December 2024,1.0,1.0,NaN,1.0,1,no,Trip Verified,Another awful experience by British Airways. T...
2,2025-03-16,Not informed,Business,Premium Economy,London to New York,January 2025,1.0,1.0,1.0,5.0,1,no,Trip Verified,"The service was rude, full of attitude to me, ..."
3,2025-03-16,Not informed,Business,Business Class,Chambery to London,January 2025,1.0,1.0,2.0,1.0,1,no,Trip Verified,This flight was a joke. There was four people ...
4,2025-03-07,Boeing 777,Business,Premium Economy,Gatwick to Antigua,March 2025,5.0,5.0,4.0,5.0,5,yes,Trip Verified,This time British Airways managed to get every...


df[['month_flown', 'year_flown']] = df['date_flown'].where(
    df['date_flown'] != 'Not informed.').str.split(" ", n=2, expand=True)

In [11]:
df['date_flown'] = pd.to_datetime(df['date_flown'], format='%B %Y', errors='coerce').dt.date

## will keep numbers instead to calculate avarage score & with NTILE() in SQL calculate portion to assign ranking

ranking_map = {
    '1': 'Very Poor',
    '2': 'Poor',
    '3': 'Neutral',
    '4': 'Good',
    '5': 'Excellent'
}

df[['seat_comfort_ranking', 'cabin_staff_service_ranking', 'food_beverages_ranking', 
    'ground_services_ranking', 'value_for_money']] = df[['seat_comfort_ranking', 
    'cabin_staff_service_ranking', 'food_beverages_ranking', 
    'ground_services_ranking', 'value_for_money']].astype(str).applymap(ranking_map.get)

In [12]:
df

,date_published,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended,verification_status,review_text_clean
0,2025-03-19,Boeing 787,Solo Leisure,Business Class,Cape Town to London,2025-03-01,5.0,2.0,5.0,4.0,4,yes,Trip Verified,Flight mainly let down by a disagreeable fligh...
1,2025-03-16,Not informed,Business,Business Class,London to Geneva,2024-12-01,1.0,1.0,NaN,1.0,1,no,Trip Verified,Another awful experience by British Airways. T...
2,2025-03-16,Not informed,Business,Premium Economy,London to New York,2025-01-01,1.0,1.0,1.0,5.0,1,no,Trip Verified,"The service was rude, full of attitude to me, ..."
3,2025-03-16,Not informed,Business,Business Class,Chambery to London,2025-01-01,1.0,1.0,2.0,1.0,1,no,Trip Verified,This flight was a joke. There was four people ...
4,2025-03-07,Boeing 777,Business,Premium Economy,Gatwick to Antigua,2025-03-01,5.0,5.0,4.0,5.0,5,yes,Trip Verified,This time British Airways managed to get every...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,2012-08-29,Not informed,Not informed,Economy Class,Not informed,NaT,2.0,3.0,1.0,NaN,3,no,Not informed,LHR-JFK-LAX-LHR. Check in was ok apart from be...
3916,2012-08-29,Not informed,Not informed,Business Class,Not informed,NaT,4.0,3.0,3.0,NaN,0,yes,Not informed,HKG-LHR in New Club World on Boeing 777-300 - ...
3917,2012-08-29,Not informed,Not informed,Premium Economy,Not informed,NaT,4.0,3.0,3.0,NaN,4,yes,Not informed,YYZ to LHR - July 2012 - I flew overnight in p...
3918,2012-08-29,Not informed,Not informed,Business Class,Not informed,NaT,4.0,3.0,2.0,NaN,3,yes,Not informed,Flew return in CW from LHR to BKK in August 20...


In [13]:
df['flight_type'] = np.where(df['route'].str.contains('via', case=False, na=False), 'layover', 'direct')
df

,date_published,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended,verification_status,review_text_clean,flight_type
0,2025-03-19,Boeing 787,Solo Leisure,Business Class,Cape Town to London,2025-03-01,5.0,2.0,5.0,4.0,4,yes,Trip Verified,Flight mainly let down by a disagreeable fligh...,direct
1,2025-03-16,Not informed,Business,Business Class,London to Geneva,2024-12-01,1.0,1.0,NaN,1.0,1,no,Trip Verified,Another awful experience by British Airways. T...,direct
2,2025-03-16,Not informed,Business,Premium Economy,London to New York,2025-01-01,1.0,1.0,1.0,5.0,1,no,Trip Verified,"The service was rude, full of attitude to me, ...",direct
3,2025-03-16,Not informed,Business,Business Class,Chambery to London,2025-01-01,1.0,1.0,2.0,1.0,1,no,Trip Verified,This flight was a joke. There was four people ...,direct
4,2025-03-07,Boeing 777,Business,Premium Economy,Gatwick to Antigua,2025-03-01,5.0,5.0,4.0,5.0,5,yes,Trip Verified,This time British Airways managed to get every...,direct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,2012-08-29,Not informed,Not informed,Economy Class,Not informed,NaT,2.0,3.0,1.0,NaN,3,no,Not informed,LHR-JFK-LAX-LHR. Check in was ok apart from be...,direct
3916,2012-08-29,Not informed,Not informed,Business Class,Not informed,NaT,4.0,3.0,3.0,NaN,0,yes,Not informed,HKG-LHR in New Club World on Boeing 777-300 - ...,direct
3917,2012-08-29,Not informed,Not informed,Premium Economy,Not informed,NaT,4.0,3.0,3.0,NaN,4,yes,Not informed,YYZ to LHR - July 2012 - I flew overnight in p...,direct
3918,2012-08-29,Not informed,Not informed,Business Class,Not informed,NaT,4.0,3.0,2.0,NaN,3,yes,Not informed,Flew return in CW from LHR to BKK in August 20...,direct


In [14]:
df[['departure', 'destination', 'layover']] = df['route'].str.extract(
    r'^(.*?)(?:\s(?:to|–|-)\s(.*?))?(?:\s(?:via|through)\s(.*))?$')

df['destination'] = df['destination'].fillna(df['layover'])
df['layover'] = df['layover'].where(df['layover'] != df['destination'], 'Not Applicable')

df.head()

,date_published,aircraft,type_of_traveller,seat_type,route,date_flown,seat_comfort_ranking,cabin_staff_service_ranking,food_beverages_ranking,ground_services_ranking,value_for_money,recommended,verification_status,review_text_clean,flight_type,departure,destination,layover
0,2025-03-19,Boeing 787,Solo Leisure,Business Class,Cape Town to London,2025-03-01,5.0,2.0,5.0,4.0,4,yes,Trip Verified,Flight mainly let down by a disagreeable fligh...,direct,Cape Town,London,NaN
1,2025-03-16,Not informed,Business,Business Class,London to Geneva,2024-12-01,1.0,1.0,NaN,1.0,1,no,Trip Verified,Another awful experience by British Airways. T...,direct,London,Geneva,NaN
2,2025-03-16,Not informed,Business,Premium Economy,London to New York,2025-01-01,1.0,1.0,1.0,5.0,1,no,Trip Verified,"The service was rude, full of attitude to me, ...",direct,London,New York,NaN
3,2025-03-16,Not informed,Business,Business Class,Chambery to London,2025-01-01,1.0,1.0,2.0,1.0,1,no,Trip Verified,This flight was a joke. There was four people ...,direct,Chambery,London,NaN
4,2025-03-07,Boeing 777,Business,Premium Economy,Gatwick to Antigua,2025-03-01,5.0,5.0,4.0,5.0,5,yes,Trip Verified,This time British Airways managed to get every...,direct,Gatwick,Antigua,NaN


In [15]:
df.to_csv('skytrax-data-clean.csv', index=False)